In [ ]:
!pip install spellchecker
!pip install unidecode
!pip install contractions
!pip install pyspellchecker
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

from wordcloud import WordCloud

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import contractions
import re
import unidecode
import nltk
nltk.download('stopwords')
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('dataset_cleaned.csv')
df_negative = df.loc[df.stars.isin([1,2]),['text_cleaned','stars']]

,text,stars,length,text_cleaned
0,I've only had food from here once and it wasn'...,1,68,food memorable panang curry balance flavor lik...
1,I will never return here again. Ever. I was ...,1,87,NOT_return ever sit booth wait dinner come scu...
2,I wish my experience was great as others. I di...,1,166,wish experience great others din wednesday nig...
3,Are the rosemary grapefruit scones supposed to...,1,81,rosemary grapefruit scone suppose taste like w...
4,Our takeout order was half wrong. Food was mis...,1,32,takeout order half wrong food miss portion siz...
...,...,...,...,...
24995,I was a loyal fan of Aroy before the ownership...,5,75,loyal fan aroy ownership change apprehensive v...
24996,Stopped here for a bite while wandering around...,5,55,stopped bite wander around faneuil hall pleasa...
24997,"A quiet place with excellent food, great music...",5,32,quiet place excellent food great music helpful...
24998,Super delicious food. Awesome vibe. I suffered...,5,41,super delicious food awesome vibe suffer disne...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(max_df=.8, min_df=.02)
matrix = vectorizer.fit_transform(df_negative['text_cleaned']).toarray()
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
columns_names = vectorizer.get_feature_names_out()
matrix_df = pd.DataFrame(matrix,columns=columns_names)

In [ ]:
matrix_df

,10,12,15,20,25,30,40,45,50,able,...,work,worst,worth,would,write,wrong,year,yelp,yes,yet
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.119721,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.414369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.143432,0.0,0.474688,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
9996,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.086631,0.0,0.0,0.118069,0.0,0.000000,0.0,0.0,0.0,0.0
9997,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.073215,0.0,0.000000,0.0,0.0,0.0,0.0
9998,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [ ]:
nmf_model = NMF(15)

doc_topic = nmf_model.fit_transform(matrix_df)
doc_topic = pd.DataFrame(doc_topic)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [ ]:
def display_topics2(model, feature_names, num_top_words, dftopic,df, topic_names=None):

    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            reviews =[]
            print("\nTopic ", ix)
           
            reviews = [ df['text'][i] for i in dftopic.nlargest(3,ix).index ]
            print("\nreview1 : ", reviews[0])
            print("\nreview2 : ",reviews[1])
            print("\nreview3 : ",reviews[2])
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [ ]:
display_topics2(nmf_model, columns_names, 5,doc_topic,df)

In [ ]:
topic_labes = {
    0 : 'manager_service',
    1 : 'general_meal_quality',
    2 : 'menu_pizza',
    3 : 'menu_chicken',
    4 : 'value_for_money',
    5 : 'waiting_time',
    6 : 'menu_burger',
    7 : 'quality_place',
    8 : 'experience',
    9 : 'drink_quality',
    10 : 'delivery_service',
    11 : 'notice_frequency',
    12 : 'general_service_quality',
    13 : 'menu_sushi',
    14 : 'overall score'
}

In [ ]:
import pickle

with open('model_loic','wb') as file:
  pickle.dump(nmf_model, file)

In [ ]:
with open('vectorizer_loic','wb') as file:
  pickle.dump(vectorizer, file)